In [1]:
!pwd

/home/jupyter


In [2]:
import os

# %cd ..
# %cd {HOME}

HOME = os.getcwd()
HOME

'/home/jupyter'

In [3]:
!git clone https://github.com/ultralytics/ultralytics
%cd ultralytics
!pip install -e .

import ultralytics

ultralytics.checks()

Ultralytics YOLOv8.0.215 🚀 Python-3.10.13 torch-1.13.1+cu117 CUDA:0 (Tesla V100-SXM2-16GB, 16161MiB)
Setup complete ✅ (4 CPUs, 14.6 GB RAM, 34.3/98.2 GB disk)


# 복구 후 압축 풀기부터 다시 시작!

In [11]:
zipPath = '{}/bbox_highway_small.zip'.format(HOME)
zipPath

'/home/jupyter/bbox_highway_small.zip'

In [ ]:
!mkdir highway

압축파일 업로드 끝난 후 압축 풀어야 함! small인데도 시간 좀 걸림!

In [15]:
!unzip {zipPath} -d highway

Archive:  /home/jupyter/bbox_highway_small.zip
  inflating: highway/bbox_highway/train/labels.cache  
  inflating: highway/bbox_highway/Suwon_CH02_20200722_1600_WED_9m_RH_highway_TW5_rainy_FHD.xml  
  inflating: highway/bbox_highway/val/labels.cache  
  inflating: highway/bbox_highway/test/labels/Suwon_CH02_20200722_1600_WED_9m_RH_highway_TW5_rainy_FHD_096.txt  
  inflating: highway/bbox_highway/val/labels/Suwon_CH02_20200722_1600_WED_9m_RH_highway_TW5_rainy_FHD_093.txt  
  inflating: highway/bbox_highway/test/labels/Suwon_CH02_20200722_1600_WED_9m_RH_highway_TW5_rainy_FHD_097.txt  
  inflating: highway/bbox_highway/val/images/Suwon_CH02_20200722_1600_WED_9m_RH_highway_TW5_rainy_FHD_093.png  
  inflating: highway/bbox_highway/val/labels/Suwon_CH02_20200722_1600_WED_9m_RH_highway_TW5_rainy_FHD_095.txt  
  inflating: highway/bbox_highway/val/images/Suwon_CH02_20200722_1600_WED_9m_RH_highway_TW5_rainy_FHD_094.png  
  inflating: highway/bbox_highway/val/labels/Suwon_CH02_20200722_1600_WED_

---

# xml 파일 내용 가지고 원하는 값으로 변환해서 리스트 생성

In [7]:
dataPath ='{}/highway/bbox_highway/'.format(HOME)
dataPath

'/home/jupyter/highway/bbox_highway/'

In [ ]:
# xml파일 생긴 모양
'''
  <image id="0" name="Suwon_CH01_20200720_1700_MON_9m_NH_highway_TW5_sunny_FHD_001.png" width="1080" height="1920">
    <box label="truck" occluded="0" xtl="862.22" ytl="420.16" xbr="910.65" ybr="467.05" z_order="27">
    </box>
    <box label="truck" occluded="0" xtl="963.68" ytl="410.17" xbr="1015.19" ybr="446.29" z_order="28">
    </box>
    <box label="truck" occluded="0" xtl="1014.42" ytl="395.56" xbr="1055.92" ybr="433.23" z_order="29">
    </box>
    <box label="truck" occluded="0" xtl="780.72" ytl="665.70" xbr="838.94" ybr="751.99" z_order="13">
    </box>
    <box label="truck" occluded="0" xtl="772.58" ytl="771.40" xbr="937.56" ybr="1009.52" z_order="12">
    </box>
    <box label="truck" occluded="0" xtl="596.20" ytl="479.35" xbr="647.76" ybr="534.69" z_order="25">
    </box>
    <box label="truck" occluded="0" xtl="524.77" ytl="489.34" xbr="567.82" ybr="551.60" z_order="24">
    </box>
    <box label="truck" occluded="0" xtl="767.99" ytl="615.10" xbr="814.67" ybr="671.39" z_order="14">
    </box>
    <box label="car" occluded="0" xtl="507.64" ytl="747.49" xbr="582.19" ybr="798.56" z_order="3">
    </box>
    ...
'''

박스는 왼쪽 위 점의 x, y 좌표 & 오른쪽 아래 점의 x, y 좌표 가 기록됨. 이걸 가지고 박스의 중심점을 찾아야 함.

## To convert MS COCO xml to YOLO V8 annotation files

In [17]:
# !pip install lxml

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 54.2 MB/s eta 0:00:0000:0100:01


In [27]:
# !pip install numpy==1.23.5

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.1/17.1 MB 67.1 MB/s eta 0:00:0000:0100:01
  Attempting uninstall: numpy
    Found existing installation: numpy 1.25.2
    Uninstalling numpy-1.25.2:
      Successfully uninstalled numpy-1.25.2


In [2]:
import numpy as np
import lxml
import os

from lxml import etree

# image 파일 내 box들의 label 3가지 종류
CLASSES = ["car", "bus", "truck"]

In [3]:
np.__version__

'1.23.5'

In [4]:
# 박스의 중심점, 가로 길이, 세로 길이 계산해서 변환
def to_yolov8(y):
  """
  # change to yolo v8 format
  # [x_top_left, y_top_left, x_bottom_right, y_bottom_right] to
  # [x_center, y_center, width, height]
  """
  width = y[2] - y[0]
  height = y[3] - y[1]

  if width < 0 or height < 0:
      print("ERROR: negative width or height ", width, height, y)
      raise AssertionError("Negative width or height")
        
  return (y[0] + (width/2)), (y[1] + (height/2)), width, height

In [10]:
# xml 파일 내용 쪼개서 지저분하게 반복되는 키 값들은 빼주고 value들만 모으기
def load_xml_annotations(f):
  tree = etree.parse(f)
  anns = []
  for dim in tree.xpath("image"):
    image_filename = dim.attrib["name"]
    width = int(dim.attrib["width"])
    height = int(dim.attrib["height"])
    # print(image_filename)
    # print(len(dim.xpath("box")))
    boxes = []
    for box in dim.xpath("box"):
      label = CLASSES.index(box.attrib["label"])
      xtl, ytl = box.attrib["xtl"], box.attrib["ytl"]
      xbr, ybr = box.attrib["xbr"], box.attrib["ybr"]

      xc, yc, w, h = to_yolov8([float(xtl), float(ytl), float(xbr), float(ybr)])  ## 박스의 중심점, 가로 길이, 세로 길이 계산해서 변환
      boxes.append([label, round(xc/width, 5), round(yc/height, 5), round(w/width, 5), round(h/height, 5)])  # 변환한 걸 그대로 리턴하지 않고 가로 길이, 세로 길이로 나눠서 스케일링 해준 값을 리턴.

    anns.append([image_filename, width, height, boxes])

  return np.array(anns, dtype = 'object')

In [14]:
# 예시로 파일 하나만 잡고 실제 객체 생성
fileName = 'Suwon_CH02_20200722_1600_WED_9m_RH_highway_TW5_rainy_FHD.xml'
label_file = os.path.join(dataPath, fileName)

anns = load_xml_annotations(label_file)

In [15]:
anns[0]

array(['Suwon_CH02_20200722_1600_WED_9m_RH_highway_TW5_rainy_FHD_001.png',
       1920, 1080,
       list([[2, 0.70172, 0.54471, 0.02857, 0.04529], [2, 0.55315, 0.86765, 0.05567, 0.10332], [2, 0.57988, 0.60055, 0.04291, 0.09265], [2, 0.64396, 0.64562, 0.0337, 0.05186], [0, 0.62288, 0.57988, 0.02068, 0.03288], [0, 0.65474, 0.7765, 0.04248, 0.08641], [0, 0.54397, 0.68912, 0.03223, 0.0533], [2, 0.71325, 0.53245, 0.03927, 0.0419], [0, 0.81297, 0.89938, 0.08124, 0.11747]])],
      dtype=object)

여기서 list에 들어간 게 아까 xml 파일에서 image 내 box 하나 하나. 가로 길이 세로 길이로 나눠준 값을 리턴했기 때문에 이미 스케일링이 되어 있는 것.

# 생성한 리스트를 가지고 폴더 생성, 해당 폴더에 txt 파일 생성

In [13]:
!pwd

/home/jupyter


In [16]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"

맨 위에 먼저 돌려놓으면 아래 코드 에러 나지 않음.

In [24]:
folderName = os.path.splitext(fileName)[0]
folderName

'Suwon_CH02_20200722_1600_WED_9m_RH_highway_TW5_rainy_FHD'

In [21]:
%cd highway/bbox_highway/
!mkdir {folderName}

[Errno 2] No such file or directory: 'highway/bbox_highway/'
/home/jupyter/highway/bbox_highway


In [30]:
thispath = os.getcwd()
print(thispath)

dirname = os.path.join(thispath, folderName)
print(dirname)

/home/jupyter/highway/bbox_highway
/home/jupyter/highway/bbox_highway/Suwon_CH02_20200722_1600_WED_9m_RH_highway_TW5_rainy_FHD


In [25]:
dataPath + folderName

'/home/jupyter/highway/bbox_highway/Suwon_CH02_20200722_1600_WED_9m_RH_highway_TW5_rainy_FHD'

In [33]:
def write_yolov8_txt(folder, annotation):
  out_filename = str(folder + annotation[0][:-3])
  out_filename = os.path.splitext(out_filename)[0] + '.txt'

  f = open(out_filename,"w+")
  for box in annotation[3]:
    f.write("{} {} {} {} {}\n".format(box[0], box[1], box[2], box[3], box[4]))

    
for ann in anns:
  write_yolov8_txt(dirname + '/', ann)

highway/bbox_highway/파일이름 -> 폴더에 잘 들어갔는지 확인.

# 트레인, 밸리드, 테스트 폴더 경로 지정

In [11]:
import os
from glob import glob

trainPath = os.path.join(dataPath,'train')
validPath = os.path.join(dataPath,'val')
testPath  = os.path.join(dataPath,'test')

trainImagesPath = os.path.join(trainPath, 'images')
validImagesPath = os.path.join(validPath, 'images')
testImagesPath  = os.path.join(testPath,  'images')

trainLabelsPath = os.path.join(trainPath, 'labels')
validLabelsPath = os.path.join(validPath, 'labels')
testLabelsPath  = os.path.join(testPath,  'labels')

In [12]:
os.makedirs(trainImagesPath, exist_ok=True)
os.makedirs(validImagesPath, exist_ok=True)
os.makedirs(testImagesPath, exist_ok=True)

os.makedirs(trainLabelsPath, exist_ok=True)
os.makedirs(validLabelsPath, exist_ok=True)
os.makedirs(testLabelsPath, exist_ok=True)

In [13]:
trainImagesPath

'/content/highway/bbox_highway/train/images'

In [15]:
for filename in glob(os.path.join(trainPath,'*.png')):
    basename = os.path.basename(filename)  # 디렉토리 경로 제외하고 파일이름만 출력
    os.rename(filename, os.path.join(trainImagesPath,basename))

for filename in glob(os.path.join(validPath,'*.png')):
    basename = os.path.basename(filename)
    os.rename(filename, os.path.join(validImagesPath,basename))

for filename in glob(os.path.join(testPath,'*.png')):
    basename = os.path.basename(filename)
    os.rename(filename, os.path.join(testImagesPath,basename))


for filename in glob(os.path.join(trainPath,'*.txt')):
    basename = os.path.basename(filename)
    os.rename(filename, os.path.join(trainLabelsPath,basename))

for filename in glob(os.path.join(validPath,'*.txt')):
    basename = os.path.basename(filename)
    os.rename(filename, os.path.join(validLabelsPath,basename))

for filename in glob(os.path.join(testPath,'*.txt')):
    basename = os.path.basename(filename)
    os.rename(filename, os.path.join(testLabelsPath,basename))

In [14]:
import torch

In [16]:
from IPython.display import Image, clear_output  # to display images

clear_output()
print(f"Setup complete. Using torch {torch.__version__} \
    ({torch.cuda.get_device_properties(0).name if torch.cuda.is_available() else 'CPU'})")

Setup complete. Using torch 2.1.0+cu118     (Tesla T4)


# Train Vehicle Detection Model

In [17]:
%cd ultralytics
!ls -al

/content/ultralytics
total 172
drwxr-xr-x 11 root root  4096 Nov 17 05:50 .
drwxr-xr-x  1 root root  4096 Nov 17 06:01 ..
-rw-r--r--  1 root root   612 Nov 17 05:46 CITATION.cff
-rw-r--r--  1 root root  5597 Nov 17 05:46 CONTRIBUTING.md
drwxr-xr-x  2 root root  4096 Nov 17 05:46 docker
drwxr-xr-x 12 root root  4096 Nov 17 05:46 docs
drwxr-xr-x  9 root root  4096 Nov 17 05:46 examples
drwxr-xr-x  8 root root  4096 Nov 17 05:46 .git
drwxr-xr-x  4 root root  4096 Nov 17 05:46 .github
-rw-r--r--  1 root root  2295 Nov 17 05:46 .gitignore
drwxr-xr-x  2 root root  4096 Nov 17 05:50 highway
drwxr-xr-x  2 root root  4096 Nov 17 05:48 .ipynb_checkpoints
-rw-r--r--  1 root root 34523 Nov 17 05:46 LICENSE
-rw-r--r--  1 root root   200 Nov 17 05:46 MANIFEST.in
-rw-r--r--  1 root root  2316 Nov 17 05:46 .pre-commit-config.yaml
-rw-r--r--  1 root root 28562 Nov 17 05:46 README.md
-rw-r--r--  1 root root 27586 Nov 17 05:46 README.zh-CN.md
-rw-r--r--  1 root root  1244 Nov 17 05:46 requirements.txt
-r

In [22]:
%cd /content/highway

/content/highway


In [23]:
from IPython.core.magic import register_line_cell_magic

# yaml에 직접 쓰기를 실행하는 함수
@register_line_cell_magic
def writetemplate(line, cell):
    with open(line, 'w') as f:
        f.write(cell.format(**globals()))

In [26]:
dataYaml='/content/highway/bbox_highway/data.yaml'

In [27]:
%%writetemplate {dataYaml}
train: /content/highway/bbox_highway/train/images
val: /content/highway/bbox_highway/val/images
test: /content/highway/bbox_highway/test/images

nc: 3
names: ['car', 'bus', 'truck']

In [ ]:
from ultralytics import YOLO

# Load a model
# model = YOLO('yolov8n.yaml')  # build a new model from YAML
# model = YOLO('yolov8n.pt')  # load a pretrained model (recommended for training)
model = YOLO('yolov8n.yaml').load('yolov8n.pt')  # build from YAML and transfer weights

# Train the model
yamlPath = '{}/highway/bbox_highway/data.yaml'.format(HOME)
results = model.train(data=yamlPath, epochs=10, imgsz=640, batch = 128, device=[0, 1])


                   from  n    params  module                                       arguments                     
  0                  -1  1       464  ultralytics.nn.modules.conv.Conv             [3, 16, 3, 2]                 
  1                  -1  1      4672  ultralytics.nn.modules.conv.Conv             [16, 32, 3, 2]                
  2                  -1  1      7360  ultralytics.nn.modules.block.C2f             [32, 32, 1, True]             
  3                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  4                  -1  2     49664  ultralytics.nn.modules.block.C2f             [64, 64, 2, True]             
  5                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  6                  -1  2    197632  ultralytics.nn.modules.block.C2f             [128, 128, 2, True]           
  7                  -1  1    295424  ultralytics.nn.modules.conv.Conv             [128

*****************************************
Setting OMP_NUM_THREADS environment variable for each process to be 1 in default, to avoid your system being overloaded, please further tune the variable for optimal performance in your application as needed. 
*****************************************


Ultralytics YOLOv8.0.215 🚀 Python-3.10.13 torch-1.13.1+cu117 CUDA:0 (Tesla V100-SXM2-16GB, 16161MiB)
                                                              CUDA:1 (Tesla V100-SXM2-16GB, 16161MiB)
WARNING ⚠️ Upgrade to torch>=2.0.0 for deterministic training.
TensorBoard: Start with 'tensorboard --logdir /home/jupyter/ultralytics/runs/detect/train3', view at http://localhost:6006/
Overriding model.yaml nc=80 with nc=3
Freezing layer 'model.22.dfl.conv.weight'
AMP: running Automatic Mixed Precision (AMP) checks with YOLOv8n...
AMP: checks passed ✅


train: Scanning /home/jupyter/highway/bbox_highway/train/labels.cache... 19775 images, 0 backgrounds, 0 corrupt: 100%|██████████| 19775/19775 [00:00<?, ?it/s]
val: Scanning /home/jupyter/highway/bbox_highway/val/labels.cache... 4176 images, 150 backgrounds, 0 corrupt: 100%|██████████| 4326/4326 [00:00<?, ?it/s]


Plotting labels to /home/jupyter/ultralytics/runs/detect/train3/labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.000714, momentum=0.9) with parameter groups 57 weight(decay=0.0), 64 weight(decay=0.001), 63 bias(decay=0.0)
Image sizes 640 train, 640 val
Using 4 dataloader workers
Logging results to /home/jupyter/ultralytics/runs/detect/train3
Starting training for 10 epochs...
Closing dataloader mosaic

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


...

       5/10      11.7G      1.179     0.8981      1.029        527        640: 100%|██████████| 155/155 [24:08<00:00,  9.35s/it]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 34/34 [04:18<00:00,  7.61s/it]


                   all       4326      64180      0.569      0.494      0.504      0.337

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


       6/10      11.7G      1.112     0.8259     0.9952        573        640: 100%|██████████| 155/155 [24:06<00:00,  9.33s/it]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 34/34 [04:24<00:00,  7.77s/it]


                   all       4326      64180      0.591      0.512      0.522      0.355

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


       7/10      10.4G      1.083     0.7995     0.9838       1092        640:  17%|█▋        | 26/155 [04:10<16:57,  7.89s/it]Traceback (most recent call last):
  File "/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/distributed.py", line 1621, in _sync_buffers
    self._sync_module_buffers(authoritative_rank)
  File "/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/distributed.py", line 1625, in _sync_module_buffers
    self._default_broadcast_coalesced(authoritative_rank=authoritative_rank)
  File "/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/distributed.py", line 1646, in _default_broadcast_coalesced
    self._distributed_broadcast_coalesced(
  File "/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/distributed.py", line 1562, in _distributed_broadcast_coalesced
    dist._broadcast_coalesced(
RuntimeError: Broken pipe

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/jupyter

In [ ]:
import os
os.getcwd()

'/content/ultralytics/ultralytics'

In [ ]:
testImages = '{}/highway/bbox_highway/test/images'.format(HOME)
from glob import glob

testList =[]
for filename in glob(os.path.join(testImages, '*.png')):
    testList.append(filename)

In [ ]:
testList

['/content/drive/MyDrive/Proj/고속도로CCTV데이터기반차량인식/data/bbox_highway/test/images/Suwon_CH02_20200722_1600_WED_9m_RH_highway_TW5_rainy_FHD_099.png',
 '/content/drive/MyDrive/Proj/고속도로CCTV데이터기반차량인식/data/bbox_highway/test/images/Suwon_CH02_20200722_1600_WED_9m_RH_highway_TW5_rainy_FHD_097.png',
 '/content/drive/MyDrive/Proj/고속도로CCTV데이터기반차량인식/data/bbox_highway/test/images/Suwon_CH02_20200722_1600_WED_9m_RH_highway_TW5_rainy_FHD_100.png',
 '/content/drive/MyDrive/Proj/고속도로CCTV데이터기반차량인식/data/bbox_highway/test/images/Suwon_CH02_20200722_1600_WED_9m_RH_highway_TW5_rainy_FHD_098.png',
 '/content/drive/MyDrive/Proj/고속도로CCTV데이터기반차량인식/data/bbox_highway/test/images/Suwon_CH02_20200722_1600_WED_9m_RH_highway_TW5_rainy_FHD_096.png']

In [ ]:
from ultralytics import YOLO

resultList =[]

# Load a pretrained YOLOv8n model
model = YOLO('/content/ultralytics/ultralytics/runs/detect/train3/weights/best.pt')

# Run inference on images
results = model(testList)

# View results
for result in results:
    resultList.append(result.cpu().numpy().boxes.cls)  # print the Boxes object containing the detection bounding boxes


0: 384x640 55 cars, 4 buss, 20 trucks, 1: 384x640 8 cars, 9 trucks, 2: 384x640 37 cars, 2 buss, 23 trucks, 3: 384x640 42 cars, 1 bus, 31 trucks, 4: 384x640 13 cars, 4 trucks, 11.4ms
Speed: 2.1ms preprocess, 2.3ms inference, 1.6ms postprocess per image at shape (1, 3, 384, 640)


In [ ]:
import pandas as pd
sum = [0, 0, 0]

for result in resultList:
    for i in range(len(sum)):
        sum[i] += result.tolist().count(i)
print(sum)

print("전체 {}장의 이미지에서 승용차 {}대, 버스 {}대, 트럭 {}대를 발견하였습니다.".format(len(resultList), sum[0], sum[1], sum[2]))

[155, 7, 87]
전체 5장의 이미지에서 승용차 155대, 버스 7대, 트럭 87대를 발견하였습니다.
